In [1]:
import os
import sys
import warnings

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Turn off oneDNN custom operations
warnings.filterwarnings('ignore', category=UserWarning)

os.chdir('..')

import json
import torch
from lightning_module import PreTrainLightningSpatial
from models import MaskedAutoencoderViT3D

from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from dataset import create_dataloaders

import numpy as np

# Set float32 matmul precision for better performance on CUDA devices with Tensor Cores
torch.set_float32_matmul_precision('medium')

In [2]:
observational_data = np.load('/home/jp4474/latent-abc-smc/data/SPATIAL/data.npy')

In [3]:
# config = {
#     "model": {
#         "name": "SpatialSIR3D",
#         "params": {
#             "decoder_depth": 6,
#             "decoder_embed_dim": 104,
#             "decoder_num_heads": 8,
#             "depth": 10,
#             "embed_dim": 128,
#             "img_size": 80,
#             "lambda_": 0.0,
#             "num_frames": 15,
#             "num_heads": 8,
#             "patch_size": 8,
#             "pred_t_dim": 15,
#             "t_patch_size": 3,
#             "z_type": "vanilla",
#             "mask_ratio": 0.25,
#         }
#     }
# }

config = {
    "model": {
        "name": "SpatialSIR3D",
        "params": {
            "decoder_depth": 8,
            "decoder_embed_dim": 104,
            "decoder_num_heads": 8,
            "depth": 10,
            "embed_dim": 144,
            "img_size": 80,
            "lambda_": 0.0,
            "num_frames": 15,
            "num_heads": 8,
            "patch_size": 8,
            "pred_t_dim": 15,
            "t_patch_size": 3,
            "z_type": "vanilla",
            "mask_ratio": 0.25,
        }
    }
}

In [4]:
# save config as yaml
with open('/home/jp4474/latent-abc-smc/tutorial_sirs/config.yaml', 'w') as f:
    json.dump(config, f, indent=4)

In [5]:
model = MaskedAutoencoderViT3D(**config['model']['params'])

img_size (80, 80) patch_size (8, 8) frames 15 t_patch_size 3
model initialized


In [6]:
dirpath = '/home/jp4474/latent-abc-smc/tutorial_sirs'

In [7]:
lightning_module = PreTrainLightningSpatial(model=model, lr=1e-3, mask_ratio=config['model']['params']['mask_ratio'])
checkpoint_callback = ModelCheckpoint(
            dirpath=dirpath,
            filename='SpatialSIR3D-{epoch:02d}-{val_loss:.4f}',
            save_top_k=3,
            monitor='val_loss',
            mode='min'
        )

lr_monitor = LearningRateMonitor(logging_interval='epoch')
early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, mode="min")

In [8]:
trainer = Trainer(
    max_epochs=100,
    accelerator='auto',
    devices=1,
    callbacks=[checkpoint_callback, lr_monitor, early_stop_callback],
    log_every_n_steps=10,
    enable_progress_bar=False,
    precision="32-true",
    fast_dev_run=False,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
train_dataloader, val_dataloader = create_dataloaders(
    data_dir='/home/jp4474/latent-abc-smc/data/SPATIAL',
    batch_size=64
)

In [ ]:
trainer.fit(lightning_module, train_dataloader, val_dataloader)

E0000 00:00:1754155419.951961 3972672 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754155419.954833 3972672 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754155419.963337 3972672 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754155419.963345 3972672 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754155419.963346 3972672 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754155419.963347 3972672 computation_placer.cc:177] computation placer already registered. Please check linka

NameError: name 'exit' is not defined

: 